In [ ]:
# if u run this notebook in collab, install this dependences

# !pip install transformers
# !pip install datasets
# !pip install wandb

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import numpy as np
from datasets import Dataset, load_dataset
import wandb
from datetime import datetime
import pandas as pd
import datasets

/home/pavel/Projects/Science/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("glue", "mrpc")

In [3]:
def preproces_dataset(raw_dataset):
  df = pd.DataFrame(raw_dataset)
  one_hot_encoded = pd.get_dummies(df['label'], prefix='label')
  df = pd.concat([df, one_hot_encoded], axis=1)
  df.drop('label', axis=1, inplace=True)
  df.drop('idx', axis=1, inplace=True)
  new_dataset = Dataset.from_pandas(df)
  return new_dataset

def preproces_dict_dataset(raw_dataset):
  train_dataset = raw_dataset['train']
  validation_dataset = raw_dataset['validation']
  test_dataset = raw_dataset['test']

  train_dataset = preproces_dataset(train_dataset)
  validation_dataset = preproces_dataset(validation_dataset)
  test_dataset = preproces_dataset(test_dataset)

  dataset = datasets.dataset_dict.DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
  })
  return dataset


In [4]:
dataset = preproces_dict_dataset(dataset)

In [5]:
labels = ['label_0', 'label_1']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [6]:
MODEL_NAME = "openai-community/gpt2-large"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [10]:
def preprocess_data(examples):
  # take a batch of texts
  text = [examples["sentence1"], examples["sentence2"]]
  # encode them
  encoding = tokenizer(*text, padding="max_length", truncation=True, max_length=128)
  n_samples, sample_len = np.shape(np.array(encoding['input_ids']))
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((n_samples, len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [11]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map: 100%|██████████| 1725/1725 [00:00<00:00, 10551.50 examples/s]


In [ ]:
# # Alternative way to preprocess data
# def tokenize_function(example):
#     return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# # here we use datacollator, that means, that batching and paddig will be applied to the dataset during the training
# tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [12]:
example = encoded_dataset['train'][0]
print(example.keys())
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
print([id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0])

dict_keys(['input_ids', 'attention_mask', 'labels'])
Amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence.Referring to him as only " the witness ", Amrozi accused his brother of deliberately distorting his evidence.[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]
[0.0, 1.0]
['label_1']


In [13]:
encoded_dataset.set_format("torch")

In [14]:
type(encoded_dataset['train']['input_ids'])

torch.Tensor

In [15]:
batch_size = 8
metric_name = "f1"

args = TrainingArguments(
    f"{MODEL_NAME}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

In [16]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

TODO: try to use raw model to predict some label

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
date = datetime.strftime(datetime.now(), "%d.%m.%Y-%H.%M.%S")
wandb.init(
    # set the wandb project where this run will be logged
    project="nlp-classifier",
    name=f"{MODEL_NAME}-cls-{date}",

    # # track hyperparameters and run metadata
    # config={
    # "learning_rate": 0.02,
    # "architecture": "CNN",
    # "dataset": "CIFAR-100",
    # "epochs": 10,
    # }
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rubikpf2002 (rubikpf2002-mipt). Use `wandb login --relogin` to force relogin


In [19]:
trainer.train()

                                                    
 10%|█         | 459/4590 [14:14<1:47:24,  1.56s/it]

{'eval_loss': 0.6859961152076721, 'eval_f1': 0.6838235294117647, 'eval_roc_auc': 0.6838235294117647, 'eval_accuracy': 0.6838235294117647, 'eval_runtime': 31.8829, 'eval_samples_per_second': 12.797, 'eval_steps_per_second': 1.6, 'epoch': 1.0}


 11%|█         | 500/4590 [15:30<2:01:57,  1.79s/it] 

{'loss': 0.6223, 'grad_norm': 6.23249626159668, 'learning_rate': 1.7821350762527233e-05, 'epoch': 1.09}


                                                    
 20%|██        | 918/4590 [28:29<1:35:22,  1.56s/it]

...

                                                     
100%|██████████| 4590/4590 [2:22:34<00:00,  1.56s/it]

{'eval_loss': 1.3407045602798462, 'eval_f1': 0.7058823529411765, 'eval_roc_auc': 0.7058823529411765, 'eval_accuracy': 0.7058823529411765, 'eval_runtime': 31.8735, 'eval_samples_per_second': 12.801, 'eval_steps_per_second': 1.6, 'epoch': 10.0}


100%|██████████| 4590/4590 [2:22:37<00:00,  1.86s/it]

{'train_runtime': 8557.8863, 'train_samples_per_second': 4.286, 'train_steps_per_second': 0.536, 'train_loss': 0.3285841972999323, 'epoch': 10.0}


TrainOutput(global_step=4590, training_loss=0.3285841972999323, metrics={'train_runtime': 8557.8863, 'train_samples_per_second': 4.286, 'train_steps_per_second': 0.536, 'total_flos': 2412728377651200.0, 'train_loss': 0.3285841972999323, 'epoch': 10.0})

In [20]:
# Закрыть сессию WandB
wandb.finish()

eval/accuracy,▁▃▃██▄▆▄▄▃
eval/f1,▁▄▄██▄▆▄▄▃
eval/loss,▂▁▁▃▃▅▆▇▇█
eval/roc_auc,▁▄▄██▄▆▄▄▃
eval/runtime,▄▄█▅▁▇▇▃▅▃
eval/samples_per_second,▅▅▁▄█▂▂▆▄▆
eval/steps_per_second,▆▆▁▃█▁▃▆▃▆
train/epoch,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇███
train/grad_norm,▁▂█▁▁▁▁▁▁
train/learning_rate,█▇▆▅▅▄▃▂▁


In [ ]:
# model.save_pretrained("./seved_model")

In [ ]:
# text = dataset['test'][18]['sentence1'] + ' ' + dataset['test'][18]['sentence2']
# true_label = [dataset['test'][18]['label_0'], dataset['test'][18]['label_1']]
# if true_label[0]:
#   true_label = 'label_0'
# else:
#   true_label = 'label_1'
# print(text)
# encoding = tokenizer(text, return_tensors="pt")
# encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
# print(encoding)
# outputs = trainer.model(**encoding)
# logits = outputs.logits
# print(logits.shape)
# sigmoid = torch.nn.Sigmoid()
# probs = sigmoid(logits.squeeze().cpu())
# print(probs)
# predictions = np.zeros(probs.shape)
# predictions[np.where(probs >= 0.5)] = 1

# # turn predicted id's into actual label names
# predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
# print(predicted_labels)
# print(true_label)

In [ ]:
# устанавливаем модель в inference режим
model.eval()

In [ ]:
test_dataset = encoded_dataset['test'] # заметим, что здесь test_dataset уже подготовлен к инференсу см. выше

In [ ]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, batch_size=16)

In [ ]:
for batch in test_dataloader:
    print(batch)
    break

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predictions = []
true_labels = []

for batch in test_dataloader:
  # Move batch data to the same device as the model
  batch = {k: v.to(model.device) for k, v in batch.items()}

  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  preds = np.argmax(logits.cpu().numpy(), axis=1)
  predictions.extend(preds)

  true_batch_labels = batch['labels'].cpu().numpy()
  true_labels.extend(true_batch_labels.T[1])  # Добавляем истинные метки для оценки

# 5. Оценка качества
f1_micro_average = f1_score(y_true=true_labels, y_pred=predictions, average='micro')
roc_auc = roc_auc_score(true_labels, predictions, average = 'micro')
accuracy = accuracy_score(true_labels, predictions)
# return as dictionary
metrics = {'f1': f1_micro_average,
            'roc_auc': roc_auc,
            'accuracy': accuracy}
report = classification_report(true_labels, predictions)

print(f"Accuracy: {accuracy}")
print(metrics)
print(report)